<a href="https://colab.research.google.com/github/OumarSatina/OpenClassroomsProjects/blob/main/emulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class Processor:
    def __init__(self):
         # Processor component initialization
        self.registers = [0] * 8  # General purpose registers
        self.accumulator = 0  # Accumulator
        self.pc = 0  # Program counter
        self.ir = 0  # Order register
        self.flags = [False] * 3  # Indicator register
        self.stack = []  # Stack of registers
        self.stack_pointer = 0  # Stack pointer
        self.memory = [0] * 256  #RAM  Blocs
        self.array = [0] * 15  # Limited size array

    def fetch(self):
        # Retrieves the instruction from memory
        self.ir = self.memory[self.pc]
        self.pc += 1

    def execute(self):
        # Decoding and executing the statement
        opcode = (self.ir & 0b1111000000000000) >> 12
        operand1 = (self.ir & 0b0000111000000000) >> 9
        operand2 = (self.ir & 0b0000000111000000) >> 6
        operand3 = self.ir & 0b0000000000111111

        old_registers = self.registers.copy()
        old_accumulator = self.accumulator
        old_pc = self.pc
        old_ir = self.ir
        old_flags = self.flags.copy()

        immediate_value = operand3
        direct_address = operand3
        register_value = self.registers[operand3]
        register_address = self.registers[operand3]
        register_indirect_address = self.memory[self.registers[operand3]]

        if opcode == 0b0000:  # MOV
            self.registers[operand1] = immediate_value
        elif opcode == 0b0001:  # ADD
            self.accumulator += register_value
        elif opcode == 0b0010:  # SUB
            self.accumulator -= self.memory[direct_address]
        elif opcode == 0b0011:  # AND
            self.accumulator &= self.memory[register_indirect_address]
        elif opcode == 0b0100:  # OR
            self.accumulator |= self.memory[direct_address]
        elif opcode == 0b0101:  # XOR
            self.accumulator ^= register_value
        elif opcode == 0b0110:  # NOT
            self.registers[operand1] = ~self.memory[direct_address]
        elif opcode == 0b0111:  # JMP
            self.pc = immediate_value

        new_registers = self.registers.copy()
        new_accumulator = self.accumulator
        new_pc = self.pc
        new_ir = self.ir
        new_flags = self.flags.copy()

        self.display_changes(old_registers, new_registers, old_accumulator, new_accumulator,
                             old_pc, new_pc, old_ir, new_ir, old_flags, new_flags)

    def display_changes(self, old_registers, new_registers, old_accumulator, new_accumulator,
                        old_pc, new_pc, old_ir, new_ir, old_flags, new_flags):
        print("Changes in values after the command is executed:")
        print("General Purpose Registries :")
        for i in range(len(old_registers)):
            if old_registers[i] != new_registers[i]:
                print(f"R{i}: {old_registers[i]} -> {new_registers[i]}")
        if old_accumulator != new_accumulator:
            print(f"Accumulateur: {old_accumulator} -> {new_accumulator}")
        if old_pc != new_pc:
            print(f"Compteur de programme: {old_pc} -> {new_pc}")
        if old_ir != new_ir:
            print(f"Registre de commandes: {old_ir} -> {new_ir}")
        print("Registre d'indicateurs:")
        for i in range(len(old_flags)):
            if old_flags[i] != new_flags[i]:
                print(f"Flag {i}: {old_flags[i]} -> {new_flags[i]}")

    def load_program(self, instructions):
        # Loads the program into memory
        for i, instruction in enumerate(instructions):
            self.memory[i] = instruction

    def load_array(self, values):
        # Loads the array into memory
        for i, value in enumerate(values):
            self.array[i] = value

    def display_registers(self):
        # Displays the current status of the registries
        print("General purpose registers:")
        for i, register in enumerate(self.registers):
            print(f"R{i}: {register}")
        print(f"Accumulator: {self.accumulator}")
        print(f"Program counter: {self.pc}")
        print(f"Order register: {self.ir}")
        print("Indicator register:")
        for i, flag in enumerate(self.flags):
            print(f"Flag {i}: {flag}")

    def display_memory(self):
        # Displays the contents of the RAM
        print("RAM contact:")
        for i, value in enumerate(self.memory):
            print(f"Adresse {i}: {value}")

    def display_array(self):
        # Displays the array contents
        print("array Content:")
        for i, value in enumerate(self.array):
            print(f"Index {i}: {value}")

    def run(self):
        # Main program execution loop
        while True:
            self.fetch()
            if self.ir == 0xFFFF:  # End of program condition (to be adapted as needed)
                break
            self.execute()

    def assemble(self, assembly_code):
        # Converts assembler instructions to machine instructions
        machine_code = []
        for line in assembly_code:
            parts = line.split()
            if parts[0] == "MOV":
                # Assemble MOV instruction
                operand1, operand2 = parts[1], parts[2]
                machine_code.append(self.assemble_mov(operand1, operand2))
            elif parts[0] == "ADD":
                # Assemble ADD instruction
                operand1, operand2 = parts[1], parts[2]
                machine_code.append(self.assemble_add(operand1, operand2))
            elif parts[0] == "SUB":
                # Assemble SUB instruction
                operand1, operand2 = parts[1], parts[2]
                machine_code.append(self.assemble_sub(operand1, operand2))
            elif parts[0] == "JMP":
                # Assemble JMP instruction
                target = int(parts[1])
                machine_code.append(self.assemble_jmp(target))
            else:
                raise ValueError(f"Unknown instruction : {parts[0]}")

        # Adds an end of program instruction
        machine_code.append(0xFFFF)

        return machine_code

    def assemble_mov(self, operand1, operand2):
        # Assemble the MOV instruction.
        opcode = 0b0000
        operand1, operand2 = self.parse_operand(operand1), self.parse_operand(operand2)
        return (opcode << 12) | (operand1 << 9) | (operand2 & 0b111111)

    def assemble_add(self, operand1, operand2):
        # Assemble the ADD instruction.
        opcode = 0b0001
        operand1, operand2 = self.parse_operand(operand1), self.parse_operand(operand2)
        return (opcode << 12) | (operand1 << 9) | (operand2 & 0b111111)

    def assemble_sub(self, operand1, operand2):
        # Assemble the SUB instruction
        opcode = 0b0010
        operand1, operand2 = self.parse_operand(operand1), self.parse_operand(operand2)
        return (opcode << 12) | (operand1 << 9) | (operand2 & 0b111111)

    def assemble_jmp(self, target):
        #Assemble the JMP instruction.
        opcode = 0b0111
        return (opcode << 12) | (target & 0b111111111111)

    def parse_operand(self, operand):
        # Analyze an operand and return its value.
        if operand.startswith('R'):
            return int(operand[1])
        elif operand.isnumeric():
            return int(operand)
        else:
            raise ValueError(f"Unknown operand : {operand}")

# Example of use
if __name__ == "__main__":
    cpu = Processor()
    assembly_code = [
        "MOV R0, 5",
        "MOV R1, 8",
        "ADD R0, R1",
        "SUB R1, 3",
        "JMP 2",
    ]
    program = cpu.assemble(assembly_code)

    cpu = Processor()
    program = [0b0000000000000001, 0b0000000000000010, 0b0000000000000011, 0xFFFF] # Example program
    cpu.load_program(program)
    array_data = [5, 8, 3, 12, 6]  # Example initial array
    cpu.load_array(array_data)
    cpu.run()
    cpu.display_registers()
    cpu.display_memory()
    cpu.display_array()




Changes in values after the command is executed:
General Purpose Registries :
R0: 0 -> 1
Registre d'indicateurs:
Changes in values after the command is executed:
General Purpose Registries :
R0: 1 -> 2
Registre d'indicateurs:
Changes in values after the command is executed:
General Purpose Registries :
R0: 2 -> 3
Registre d'indicateurs:
General purpose registers:
R0: 3
R1: 0
R2: 0
R3: 0
R4: 0
R5: 0
R6: 0
R7: 0
Accumulator: 0
Program counter: 4
Order register: 65535
Indicator register:
Flag 0: False
Flag 1: False
Flag 2: False
RAM contact:
Adresse 0: 1
Adresse 1: 2
Adresse 2: 3
Adresse 3: 65535
Adresse 4: 0
Adresse 5: 0
Adresse 6: 0
Adresse 7: 0
Adresse 8: 0
Adresse 9: 0
Adresse 10: 0
Adresse 11: 0
Adresse 12: 0
Adresse 13: 0
Adresse 14: 0
Adresse 15: 0
Adresse 16: 0
Adresse 17: 0
Adresse 18: 0
Adresse 19: 0
Adresse 20: 0
Adresse 21: 0
Adresse 22: 0
Adresse 23: 0
Adresse 24: 0
Adresse 25: 0
Adresse 26: 0
Adresse 27: 0
Adresse 28: 0
Adresse 29: 0
Adresse 30: 0
Adresse 31: 0
Adresse 32: 0

In [ ]:
class Processor:
    def __init__(self):
        # Processor component initialization
        self.registers = [0] * 8  # General-purpose registers
        self.accumulator = 0  # Accumulator
        self.pc = 0  # Program counter
        self.ir = 0  # Instruction register
        self.flags = [False] * 3  # Flag register
        self.stack = []  # Register stack
        self.stack_pointer = 0  # Stack pointer
        self.memory = [0] * 256  # RAM blocks
        self.array = [0] * 15  # Limited size array

    def fetch(self):
        # Fetches instruction from memory
        self.ir = self.memory[self.pc]
        self.pc += 1

    def execute(self):
        # Decodes and executes instruction
        opcode = (self.ir & 0b1111000000000000) >> 12
        operand1 = (self.ir & 0b0000111000000000) >> 9
        operand2 = (self.ir & 0b0000000111000000) >> 6
        operand3 = self.ir & 0b0000000000111111

        old_registers = self.registers.copy()
        old_accumulator = self.accumulator
        old_pc = self.pc
        old_ir = self.ir
        old_flags = self.flags.copy()

        immediate_value = operand3
        direct_address = operand3
        register_value = self.registers[operand3]
        register_address = self.registers[operand3]
        register_indirect_address = self.memory[self.registers[operand3]]

        if opcode == 0b0000:  # MOV
            self.registers[operand1] = immediate_value
        elif opcode == 0b0001:  # ADD
            self.accumulator += register_value
        elif opcode == 0b0010:  # SUB
            self.accumulator -= self.memory[direct_address]
        elif opcode == 0b0011:  # AND
            self.accumulator &= self.memory[register_indirect_address]
        elif opcode == 0b0100:  # OR
            self.accumulator |= self.memory[direct_address]
        elif opcode == 0b0101:  # XOR
            self.accumulator ^= register_value
        elif opcode == 0b0110:  # NOT
            self.registers[operand1] = ~self.memory[direct_address]
        elif opcode == 0b0111:  # JMP
            self.pc = immediate_value

        new_registers = self.registers.copy()
        new_accumulator = self.accumulator
        new_pc = self.pc
        new_ir = self.ir
        new_flags = self.flags.copy()

        self.display_changes(old_registers, new_registers, old_accumulator, new_accumulator,
                             old_pc, new_pc, old_ir, new_ir, old_flags, new_flags)

    def display_changes(self, old_registers, new_registers, old_accumulator, new_accumulator,
                        old_pc, new_pc, old_ir, new_ir, old_flags, new_flags):
        print("Value changes after executing the command:")
        print("General-purpose registers:")
        for i in range(len(old_registers)):
            if old_registers[i] != new_registers[i]:
                print(f"R{i}: {old_registers[i]} -> {new_registers[i]}")
        if old_accumulator != new_accumulator:
            print(f"Accumulator: {old_accumulator} -> {new_accumulator}")
        if old_pc != new_pc:
            print(f"Program counter: {old_pc} -> {new_pc}")
        if old_ir != new_ir:
            print(f"Instruction register: {old_ir} -> {new_ir}")
        print("Flag register:")
        for i in range(len(old_flags)):
            if old_flags[i] != new_flags[i]:
                print(f"Flag {i}: {old_flags[i]} -> {new_flags[i]}")

    def load_program(self, instructions):
        # Loads the program into memory
        for i, instruction in enumerate(instructions):
            self.memory[i] = instruction

    def load_array(self, values):
        # Loads the array into memory
        for i, value in enumerate(values):
            self.array[i] = value

    def display_registers(self):
        # Displays the current register state
        print("General-purpose registers:")
        for i, register in enumerate(self.registers):
            print(f"R{i}: {register}")
        print(f"Accumulator: {self.accumulator}")
        print(f"Program counter: {self.pc}")
        print(f"Instruction register: {self.ir}")
        print("Flag register:")
        for i, flag in enumerate(self.flags):
            print(f"Flag {i}: {flag}")

    def display_memory(self):
        # Displays the RAM contents
        print("RAM content:")
        for i, value in enumerate(self.memory):
            print(f"Address {i}: {value}")

    def display_array(self):
        # Displays the array content
        print("Array content:")
        for i, value in enumerate(self.array):
            print(f"Index {i}: {value}")

    def run(self):
        # Main program execution loop
        while True:
            self.fetch()
            if self.ir == 0xFFFF:  # Program termination condition (to be adapted as needed)
                break
            self.execute()

    def assemble(self, assembly_code):
        # Converts assembly instructions to machine instructions
        machine_code = []
        for line in assembly_code:
            parts = line.split()
            if parts[0] == "MOV":
                # Assemble MOV instruction
                operand1, operand2 = parts[1], parts[2]
                machine_code.append(self.assemble_mov(operand1, operand2))
            elif parts[0] == "ADD":
                # Assemble ADD instruction
                operand1, operand2 = parts[1], parts[2]
                machine_code.append(self.assemble_add(operand1, operand2))
            elif parts[0] == "SUB":
                # Assemble SUB instruction
                operand1, operand2 = parts[1], parts[2]
                machine_code.append(self.assemble_sub(operand1, operand2))
            elif parts[0] == "JMP":
                # Assemble JMP instruction
                target = int(parts[1])
                machine_code.append(self.assemble_jmp(target))
            else:
                raise ValueError(f"Unknown instruction: {parts[0]}")

        # Add a program end instruction
        machine_code.append(0xFFFF)

        return machine_code

    def assemble_mov(self, operand1, operand2):
        # Assembles MOV instruction
        opcode = 0b0000
        operand1, operand2 = self.parse_operand(operand1), self.parse_operand(operand2)
        return (opcode << 12) | (operand1 << 9) | (operand2 & 0b111111)

    def assemble_add(self, operand1, operand2):
        # Assembles ADD instruction
        opcode = 0b0001
        operand1, operand2 = self.parse_operand(operand1), self.parse_operand(operand2)
        return (opcode << 12) | (operand1 << 9) | (operand2 & 0b111111)

    def assemble_sub(self, operand1, operand2):
        # Assembles SUB instruction
        opcode = 0b0010
        operand1, operand2 = self.parse_operand(operand1), self.parse_operand(operand2)
        return (opcode << 12) | (operand1 << 9) | (operand2 & 0b111111)

    def assemble_jmp(self, target):
        # Assembles JMP instruction
        opcode = 0b0111
        return (opcode << 12) | (target & 0b111111111111)

    def parse_operand(self, operand):
        # Parses an operand and returns its value
        if operand.startswith('R'):
            return int(operand[1])
        elif operand.isnumeric():
            return int(operand)
        else:
            raise ValueError(f"Unknown operand: {operand}")

# Example usage
if __name__ == "__main__":
    cpu = Processor()
    assembly_code = [
        "MOV R0, 5",
        "MOV R1, 8",
        "ADD R0, R1",
        "SUB R1, 3",
        "JMP 2",
    ]
    program = cpu.assemble(assembly_code)

    cpu.load_program(program)
    array_data = [5, 8, 3, 12, 6]  # Example initial array
    cpu.load_array(array_data)
    cpu.run()
    cpu.display_registers()
    cpu.display_memory()
    cpu.display_array()


Value changes after executing the command:
General-purpose registers:
R0: 0 -> 5
Flag register:


IndexError: ignored

In [ ]:

class Processor:
    def __init__(self):
        # Initialise les registres
        self.registers = [0] * 8
        self.accumulator = 0
        self.pc = 0
        self.ir = 0
        self.flags = [False] * 3

    def fetch(self):
        # Récupère l'instruction en cours d'exécution
        self.ir = self.memory[self.pc]
        self.pc += 1

    def execute(self):
        # Exécute l'instruction en cours d'exécution
        opcode = (self.ir & 0b1111000000000000) >> 12
        operand1 = (self.ir & 0b0000111000000000) >> 9
        operand2 = self.ir & 0b00000001111111

        if opcode == 0b0000:  # MOV
            self.registers[operand1] = operand2
        elif opcode == 0b0001:  # ADD
            self.accumulator += self.registers[operand1]
        elif opcode == 0b0010:  # SUB
            self.accumulator -= self.registers[operand1]
        elif opcode == 0b0011:  # AND
            self.accumulator &= self.registers[operand1]
        elif opcode == 0b0100:  # OR
            self.accumulator |= self.registers[operand1]
        elif opcode == 0b0101:  # XOR
            self.accumulator ^= self.registers[operand1]
        elif opcode == 0b0110:  # NOT
            self.registers[operand1] = ~self.registers[operand1]
        elif opcode == 0b0111:  # JMP
            self.pc = operand2

    def load_program(self, program):
        # Charge le programme dans la mémoire
        for i in range(len(program)):
            self.memory[i] = program[i]

    def display_registers(self):
        # Affiche les valeurs des registres
        print("Registres :")
        for i in range(len(self.registers)):
            print(f"R{i}: {self.registers[i]}")

    def display_memory(self):
        # Affiche le contenu de la mémoire
        print("Mémoire :")
        for i in range(len(self.memory)):
            print(f"Adresse {i}: {self.memory[i]}")

    def run(self):
        # Exécute le programme jusqu'à ce qu'il atteigne la fin
        while self.ir != 0xFFFF:
            self.fetch()
            self.execute()

    def assemble(self, assembly_code):
        # Convertit le code assembleur en code machine
        machine_code = []
        for line in assembly_code:
            parts = line.split()
            if parts[0] == "MOV":
                # Assemble MOV instruction
                operand1, operand2 = parts[1], parts[2]
                machine_code.append(self.assemble_mov(operand1, operand2))
            elif parts[0] == "ADD":
                # Assemble ADD instruction
                operand1, operand2 = parts[1], parts[2]
                machine_code.append(self.assemble_add(operand1, operand2))
            elif parts[0] == "SUB":
                # Assemble SUB instruction
                operand1, operand2 = parts[1], parts[2]
                machine_code.append(self.assemble_sub(operand1, operand2))
            elif parts[0] == "JMP":
                # Assemble JMP instruction
                target = int(parts[1])
                machine_code.append(self.assemble_jmp(target))
            else:
                raise ValueError(f"Unknown instruction : {parts[0]}")

        # Adds an end of program instruction
        machine_code.append(0xFFFF)

        return machine_code

